In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [22]:
lstm = nn.LSTM(3, 3)
inputs = [torch.randn(1, 3) for _ in range(5)]

hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)

inputs = torch.cat(inputs).view(len(inputs), 1, -1)

hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.2932, -0.0950, -0.0214]],

        [[-0.1582, -0.3642, -0.0805]],

        [[-0.0498, -0.2909, -0.0352]],

        [[ 0.0043, -0.6981, -0.0893]],

        [[-0.0660, -0.4401, -0.1084]]])
(tensor([[[-0.0660, -0.4401, -0.1084]]]), tensor([[[-0.2149, -1.2208, -0.3779]]]))


In [81]:
class LSTMNumSeq(nn.Module):
    
    def __init__(self,embedding_dim, hidden_dim):
        super(LSTMNumSeq, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(9, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, 1)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, seq):
        embeds = self.word_embeddings(seq)
        lstm_out, self.hidden = self.lstm(embeds.view(1, 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(1, -1))
        out_seq = F.log_softmax(tag_space, dim=1)
        return out_seq
        

In [82]:
input_seq = [("012345678","9"),("123456789","0"),("234567890","1"),("345678901","2"),("456789012","3")]
num_to_ix = {"0": 0, "1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9}

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[seq[w]] for w in range(len(seq))]
    return torch.tensor(idxs, dtype = torch.long)


In [83]:
model = LSTMNumSeq(6, 6)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

with torch.no_grad():
    inputs = prepare_sequence(input_seq[0][0], num_to_ix)
    output_seq = model(inputs)
    print(output_seq)
    
for epoch in range(300):
    for sequence, ty in input_seq:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        seq_in = prepare_sequence(sequence, num_to_ix)
        target = prepare_sequence(ty, num_to_ix)
        
        out = model(seq_in)
        
        loss = loss_function(out, target)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    inputs = prepare_sequence(input_seq[0][0], num_to_ix)
    output_seq = model(inputs)
    print(output_seq)

RuntimeError: input.size(-1) must be equal to input_size. Expected 6, got 54